In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from copulas.datasets import sample_trivariate_xyz
from copulas.visualization import compare_3d
from sklearn.impute import SimpleImputer

# Función para limpiar texto en el dataframe
def clean_text(df, sequences_to_remove=['*', '**']):
    for column in df.columns:
        if df[column].dtype == 'object':
            for sequence in sequences_to_remove:
                df[column] = df[column].apply(lambda x: x.replace(sequence, '') if isinstance(x, str) and sequence in x else x)
    return df

# Función para convertir columnas a tipo numérico o usar LabelEncoder
def convert_to_numeric(df):
    le_dict = {}
    for column in df.columns:
        if df[column].dtype in ['object', 'category']:
            contains_letters = any(isinstance(val, str) and any(c.isalpha() for c in val) for val in df[column])
            if not contains_letters:
                df[column] = pd.to_numeric(df[column], errors='coerce')
            else:
                le = LabelEncoder()
                df[column] = le.fit_transform(df[column].astype(str))
                le_dict[column] = le  # Guardar el LabelEncoder para uso posterior
    return df, le_dict

# Función para cargar y limpiar datos
def load_and_clean_data(file_path):
    df = pd.read_excel(file_path)
    df = clean_text(df)
    df, le_dict = convert_to_numeric(df)
    return df, le_dict

# Crear una función para asignar los grupos según la mediana de Omega score
def asignar_grupo(omega_score, mediana):
    return 0 if omega_score < mediana else 1

# Cargar y limpiar los datos
file_path = "D:/MÁSTER DATA SCIENCE/KSCHOOL/1.TFM/PARTTE 2 TFM/df6.xlsx"
df_cleaned_data, le_dict = load_and_clean_data(file_path)

# Eliminar columnas no necesarias
df_cleaned_data = df_cleaned_data.drop(columns=['Patient ID #', 'Sample ID #'])

# Verificar los datos limpiados
print(df_cleaned_data.head())
print(df_cleaned_data.info())

# Formar los grupos en función de 'Omega score'
mediana_omega_score = df_cleaned_data['Omega score'].median()

# Aplicar la función al dataframe para crear la columna 'Grupo'
df_cleaned_data['Grupo'] = df_cleaned_data['Omega score'].apply(lambda x: asignar_grupo(x, mediana_omega_score))

# Verificar la asignación de grupos
print(df_cleaned_data[['Tumor type', 'Omega score', 'Grupo']].head())

# Dividir los datos en función del grupo
X = df_cleaned_data.drop(columns=['Tumor type'])
y = df_cleaned_data['Tumor type']

# Crear DataFrames para resultados
table_results_df = pd.DataFrame(columns=['Model', 'Set', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Global Score'])

# Función para preprocesar y entrenar los modelos
def preprocess_and_train(X, y, table_results_df, filename):
    # Generar datos sintéticos usando copulas
    synthetic_data = sample_trivariate_xyz(len(X))
    
    # Asegurarse de que synthetic_data tenga la columna 'Tumor type'
    synthetic_data['Tumor type'] = y
    
    # Verificar que todas las columnas requeridas estén presentes en synthetic_data
    required_columns = list(X.columns) + ['Tumor type']
    for column in required_columns:
        if column not in synthetic_data.columns:
            raise KeyError(f"'{column}' no está presente en los datos sintéticos generados.")
    
    # Concatenar los datos originales y sintéticos
    X_resampled = pd.concat([X, synthetic_data.drop(columns=['Tumor type'])], axis=1)
    y_resampled = synthetic_data['Tumor type']
    
# Inicializar los modelos
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'KNN': KNeighborsClassifier(),
    }

    for model_name, model in models.items():
        try:
            # Cross validation
            cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
            scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=1)
            print(f"Cross-validated accuracy for {model_name}: {scores.mean():.4f}")

            # Entrenar el modelo
            model.fit(X_train, y_train)

            # Evaluar en el conjunto de prueba
            y_test_pred = model.predict(X_test)
            report = classification_report(y_test, y_test_pred, output_dict=True)
            print(f"Classification report for {model_name} on test data:\n", classification_report(y_test, y_test_pred))

            # Guardar los resultados
            table_results_df = pd.concat([table_results_df, pd.DataFrame({
                'Model': model_name,
                'Set': 'Test',
                'Accuracy': accuracy_score(y_test, y_test_pred),
                'Precision': precision_score(y_test, y_test_pred, average='weighted'),
                'Recall': recall_score(y_test, y_test_pred, average='weighted'),
                'F1-Score': f1_score(y_test, y_test_pred, average='weighted'),
                'Global Score': scores.mean()
            }, index=[0])], ignore_index=True)
        
        except Exception as e:
            print(f"Error al entrenar o evaluar el modelo {model_name}: {e}")

    table_results_df.to_excel(filename, index=True)
    
    return X_resampled, y_resampled

# Cargar y limpiar los datos
file_path = "D:/MÁSTER DATA SCIENCE/KSCHOOL/1.TFM/PARTTE 2 TFM/df6.xlsx"
df_cleaned_data, le_dict = load_and_clean_data(file_path)

# Eliminar columnas no necesarias
df_cleaned_data = df_cleaned_data.drop(columns=['Patient ID #', 'Sample ID #'])

# Formar los grupos en función de 'Omega score'
mediana_omega_score = df_cleaned_data['Omega score'].median()

# Aplicar la función al dataframe para crear la columna 'Grupo'
df_cleaned_data['Grupo'] = df_cleaned_data['Omega score'].apply(lambda x: asignar_grupo(x, mediana_omega_score))

# Verificar la asignación de grupos
print(df_cleaned_data[['Tumor type', 'Omega score', 'Grupo']].head())

# Dividir los datos en función del grupo
X = df_cleaned_data.drop(columns=['Tumor type'])
y = df_cleaned_data['Tumor type']

# Crear DataFrames para resultados
table_results_df = pd.DataFrame(columns=['Model', 'Set', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Global Score'])

# Entrenar modelos en el conjunto completo
preprocess_and_train(X, y, table_results_df, 'D:/MÁSTER DATA SCIENCE/KSCHOOL/1.TFM/PARTTE 2 TFM/table_results_df.xlsx')

# Visualizar en 3D con copulas
# Asegúrate de proporcionar los datos adecuados a compare_3d para la visualización
compare_3d(data, 'D:/MÁSTER DATA SCIENCE/KSCHOOL/1.TFM/PARTTE 2 TFM/copulas_3d_comparison.png')

# Visualizar uno de los árboles individuales del Random Forest
feature_names = X.columns
class_names = ['Grupo 0', 'Grupo 1']

# Seleccionar uno de los estimadores (árboles) del Random Forest
estimator = models['Random Forest'].estimators_[0]

# Crear la visualización del árbol
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5,5), dpi=80)
plot_tree(estimator,
          feature_names=feature_names,
          class_names=class_names,
          filled=True)
fig.savefig('D:/MÁSTER DATA SCIENCE/KSCHOOL/1.TFM/PARTTE 2 TFM/rf_individualtree.png')

plt.show()

# Análisis de subgrupos dentro del mismo dataframe
# Crear una función para realizar el análisis de subgrupos
def analizar_subgrupos(df, grupo, tumor_types):
    resultados = {}
    for tumor in tumor_types:
        subgrupo = df[(df['Grupo'] == grupo) & (df['Tumor type'] == tumor)]
        if not subgrupo.empty:
            X_subgrupo = subgrupo.drop(columns=['Tumor type', 'Grupo'])
            y_subgrupo = subgrupo['Grupo']
            X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_subgrupo, y_subgrupo, test_size=0.3, random_state=42)
            dt_model_sub = DecisionTreeClassifier(random_state=42)
            dt_model_sub.fit(X_train_sub, y_train_sub)
            y_pred_sub = dt_model_sub.predict(X_test_sub)
            reporte = classification_report(y_test_sub, y_pred_sub, output_dict=True)
            resultados[tumor] = reporte['weighted avg']
    return resultados

# Realizar el análisis para el Grupo 1
tumor_types = df_cleaned_data['Tumor type'].unique()
resultados_grupo_1 = analizar_subgrupos(df_cleaned_data, grupo=1, tumor_types=tumor_types)

# Mostrar los resultados del análisis de subgrupos
for tumor, resultado in resultados_grupo_1.items():
    print(f"Resultados para {tumor}:")
    print(resultado)

# Crear gráfico de barras para los resultados
def plot_subgroup_results(resultados):
    metrics = ['precision', 'recall', 'f1-score']
    tumor_names = list(resultados.keys())
    
    data = {metric: [resultados[tumor][metric] for tumor in tumor_names] for metric in metrics}
    
    x = np.arange(len(tumor_names))
    width = 0.25
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    bars1 = ax.bar(x - width, data['precision'], width, label='Precision')
    bars2 = ax.bar(x, data['recall'], width, label='Recall')
    bars3 = ax.bar(x + width, data['f1-score'], width, label='F1-score')
    
    ax.set_xlabel('Tumor type')
    ax.set_ylabel('Scores')
    ax.set_title('Precision, Recall, and F1-score by Tumor type for Group 1')
    ax.set_xticks(x)
    ax.set_xticklabels(tumor_names, rotation=45, ha='right')
    ax.legend()

    fig.tight_layout()
    plt.show()

plot_subgroup_results(resultados_grupo_1)

# Inversa del LabelEncoder
for column, le in le_dict.items():
    df_cleaned_data[column] = le.inverse_transform(df_cleaned_data[column])

print("DataFrame con los datos decodificados:")
print(df_cleaned_data)


   Unnamed: 0  Tumor type  AJCC Stage  AFP (pg/ml)  Angiopoietin-2 (pg/ml)  \
0           0           1           0     1583.450                 5598.50   
1           1           1           0      715.308                20936.35   
2           2           1           1     4365.530                 2350.93   
3           3           1           1      715.308                 1604.34   
4           4           1           1      801.300                 2087.57   

   AXL (pg/ml)  CA-125 (U/ml)  CA 15-3 (U/ml)  CA19-9 (U/ml)  CD44 (ng/ml)  \
0      3621.04          5.090           19.08         16.452          9.81   
1      2772.96          7.270           10.04         40.910         27.57   
2      4120.77          4.854           16.96         16.452         14.59   
3      2029.96          5.390            8.31         16.452          7.78   
4      2069.17          4.854           11.73         16.452         12.21   

   ...  sHER2/sEGFR2/sErbB2 (pg/ml)  sPECAM-1 (pg/ml)  TGFa (p

KeyError: "'Unnamed: 0' no está presente en los datos sintéticos generados."